<table style="border: none" align="left">
   <tr style="border: none">
       <tr><font face="verdana" size="2" color="black">Updated: benoit.marolleau@fr.ibm.com: Demo Version with Model creation through PySpark and/or by creating a dataset + Automatic Model Builder wizard
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predict outdoor equipment purchase with IBM Watson Machine Learning</b></tr>
      
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to get data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0 and Apache® Spark 2.0.

You will use a publicly available data set, **GoSales Transactions for Naive Bayes Model**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment, and others.

**Note**: In this notebook we use GOSales data available on [Data Science Experience Community](https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600).

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Upload **GoSales Transactions** data as a data asset in IBM Data Science Experience.
-  Make sure that you are using a Spark 2.0 kernel.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install wget --user <BR>

### 2.1: Import external data - wget 

In this subsection you will download raw data and create your dataset

In [13]:
!pip install wget 
import wget

link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
filename = wget.download(link_to_data)
print filename

GoSales_Tx_NaiveBayes.csv


In [12]:
#Cleanup GPFS
#!find /gpfs/global_fs01/sym_shared/YPProdSpark/user/s4b5-8af1a9682f921a-a614a6ea83ce/ -name GoSales_Tx_NaiveBayes.csv
!rm /gpfs/global_fs01/sym_shared/YPProdSpark/user/s4b5-8af1a9682f921a-a614a6ea83ce/notebook/work/GoSales_Tx_NaiveBayes.csv*
!find /gpfs/global_fs01/sym_shared/YPProdSpark/user/s4b5-8af1a9682f921a-a614a6ea83ce/ -name metastore_db
!ls -l /gpfs/global_fs01/sym_shared/YPProdSpark/user/s4b5-8af1a9682f921a-a614a6ea83ce/notebook/jupyter-rt/kernel-23c975ff-bd71-454a-a3c7-ade67d9c6ff3-20180511_120950/metastore_db
!rm /gpfs/global_fs01/sym_shared/YPProdSpark/user/s4b5-8af1a9682f921a-a614a6ea83ce/notebook/jupyter-rt/kernel-23c975ff-bd71-454a-a3c7-ade67d9c6ff3-20180511_120950/metastore_db/db.lck


/gpfs/global_fs01/sym_shared/YPProdSpark/user/s4b5-8af1a9682f921a-a614a6ea83ce/notebook/jupyter-rt/kernel-23c975ff-bd71-454a-a3c7-ade67d9c6ff3-20180511_120950/metastore_db
total 24
-rw------- 1 s4b5-8af1a9682f921a-a614a6ea83ce users    4 May 11 07:37 dbex.lck
drwx------ 2 s4b5-8af1a9682f921a-a614a6ea83ce users 4096 May 11 07:37 log
-rw------- 1 s4b5-8af1a9682f921a-a614a6ea83ce users  608 May 11 07:37 README_DO_NOT_TOUCH_FILES.txt
drwx------ 2 s4b5-8af1a9682f921a-a614a6ea83ce users 8192 May 11 07:37 seg0
-rw------- 1 s4b5-8af1a9682f921a-a614a6ea83ce users 1050 May 11 07:37 service.properties
drwx------ 2 s4b5-8af1a9682f921a-a614a6ea83ce users 4096 May 11 08:02 tmp
rm: cannot remove ‘/gpfs/global_fs01/sym_shared/YPProdSpark/user/s4b5-8af1a9682f921a-a614a6ea83ce/notebook/jupyter-rt/kernel-23c975ff-bd71-454a-a3c7-ade67d9c6ff3-20180511_120950/metastore_db/db.lck’: No such file or directory


The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [14]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [15]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [35]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [16]:
print "Number of records: " + str(df_data.count())

Number of records: 60252


As you can see, the data set contains 60252 records.

### 2.2: Import external data - Connection your existing database: Db2 for i  (requires Db2 Connect :o)

In this subsection you will download raw data from a Connection and create your dataset

In [ ]:

# @hidden_cell    
# DB2 LUW Standard driver in that current code version & ingest.Connectors => Requires Db2 Connect licenses :o
credentials_1 = {
  'database':'T01FF3B4',
  'password':"""******""",
  'sg_service_url':'https://sgmanager.ng.bluemix.net',
  'port':'16987',
  'host':'cap-sg-prd-1.integration.ibmcloud.com',
  'username':'benoit'
}
from ingest.Connectors import Connectors
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

DB2loadOptions = {
                     Connectors.DB2.HOST              : 'cap-sg-prd-1.integration.ibmcloud.com',
                     Connectors.DB2.PORT              : '16987',
                     Connectors.DB2.DATABASE          : 'T01FF3B4',
                     Connectors.DB2.USERNAME          : 'benoit',
                     Connectors.DB2.PASSWORD          : 'abc123',
                      Connectors.DB2.SOURCE_TABLE_NAME         : 'SALES'}

DB2DF = sqlContext.read.format("com.ibm.spark.discover").options(**DB2loadOptions).load()
DB2DF.printSchema()
DB2DF.show()

### 2.3: Import data - From your CSV Data Asset based on your Db2 for i  database (Connection or import)

In this subsection you will download raw data from COS and create your dataset

In [24]:
# Db2 for i RAW DATA - Imported Table:  SALES

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'QYJcDPVVfRe8GR_5ES5iqcrNu2a_duOT2CeSov9nWEMR',
    'service_id': 'iam-ServiceId-58078be1-3843-4dff-bda3-dcef403bb10f',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_63432e8c7d8140b184e5b51bbc9f7f01_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df_data_2 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('GoSales_Tx_NaiveBayes_IBMi.csv', 'projectibmib44b9b30d9304b75bfe3d588ecd1b28f'))
df_data_2.show()


+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [26]:
#Db2 for i - Refined Data (cleanup, indexing) with Data Refinery  - Table: SALES
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('data_asset/GoSales_Tx_NaiveBayes_IBMi.csv_shaped_K4VhCSdTSCCXUoCzCn5C8w.csv', 'projectibmib44b9b30d9304b75bfe3d588ecd1b28f'))
df_data_1.take(5)


[Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'M', AGE=u'27', MARITAL_STATUS=u'Single', PROFESSION=u'Professional', MARITAL_STATUS_1=u'Single', MARITAL_STATUS_2=u'0', MARITAL_STATUS_IDX=u'0', GENDER_1=u'0', GENDER_IDX=u'0'),
 Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'F', AGE=u'39', MARITAL_STATUS=u'Married', PROFESSION=u'Other', MARITAL_STATUS_1=u'1', MARITAL_STATUS_2=u'1', MARITAL_STATUS_IDX=u'1', GENDER_1=u'F', GENDER_IDX=u'1'),
 Row(PRODUCT_LINE=u'Mountaineering Equipment', GENDER=u'F', AGE=u'39', MARITAL_STATUS=u'Married', PROFESSION=u'Other', MARITAL_STATUS_1=u'1', MARITAL_STATUS_2=u'1', MARITAL_STATUS_IDX=u'1', GENDER_1=u'F', GENDER_IDX=u'1'),
 Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'F', AGE=u'56', MARITAL_STATUS=u'Unspecified', PROFESSION=u'Hospitality', MARITAL_STATUS_1=u'Unspecified', MARITAL_STATUS_2=u'Unspecified', MARITAL_STATUS_IDX=u'2', GENDER_1=u'F', GENDER_IDX=u'1'),
 Row(PRODUCT_LINE=u'Golf Equipment', GENDER=u'M', AGE=u'45', MARITAL_STATUS=u'M

In [27]:
df_data_1.show()

+--------------------+------+---+--------------+------------+----------------+----------------+------------------+--------+----------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|MARITAL_STATUS_1|MARITAL_STATUS_2|MARITAL_STATUS_IDX|GENDER_1|GENDER_IDX|
+--------------------+------+---+--------------+------------+----------------+----------------+------------------+--------+----------+
|Personal Accessories|     M| 27|        Single|Professional|          Single|               0|                 0|       0|         0|
|Personal Accessories|     F| 39|       Married|       Other|               1|               1|                 1|       F|         1|
|Mountaineering Eq...|     F| 39|       Married|       Other|               1|               1|                 1|       F|         1|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|     Unspecified|     Unspecified|                 2|       F|         1|
|      Golf Equipment|     M| 45|       Married|     Re

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [21]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print "Number of training records: " + str(train_data.count())
print "Number of testing records : " + str(test_data.count())
print "Number of prediction records : " + str(predict_data.count())

Number of training records: 48176
Number of testing records : 10860
Number of prediction records : 1216


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In [ ]:
#In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [32]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [33]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [37]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_data) for column in list(set(df_data.columns)-set(['date'])) ]

pipeline2 = Pipeline(stages=indexers)
df_r = pipeline2.fit(df_data).transform(df_data)
print "Number of prediction records with Indexing : " + str(df_r.count())
print df_r.printSchema()



Number of prediction records with Indexing : 60252
root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)
 |-- GENDER_index: double (nullable = true)
 |-- PRODUCT_LINE_index: double (nullable = true)
 |-- MARITAL_STATUS_index: double (nullable = true)
 |-- PROFESSION_index: double (nullable = true)
 |-- AGE_index: double (nullable = true)

None


In the following step, create a feature vector by combining all features together.

### 3.1.bis: Export prepared (indexed) dataset to a CSV file (Cloud object storage)

In this subsection you will export your dataset in COS using a S3 connector. This data asset can be used to create a Model using the Automatic Model Builder 

In [39]:
from ingest.Connectors import Connectors

S3saveoptions = {
      Connectors.BluemixCloudObjectStorage.URL                      : "https://s3-api.us-geo.objectstorage.service.networklayer.com",
      Connectors.BluemixCloudObjectStorage.IAM_URL                  : "https://iam.ng.bluemix.net/oidc/token",
      Connectors.BluemixCloudObjectStorage.RESOURCE_INSTANCE_ID     : "iam-ServiceId-58078be1-3843-4dff-bda3-dcef403bb10f",
      Connectors.BluemixCloudObjectStorage.API_KEY                  : "QYJcDPVVfRe8GR_5ES5iqcrNu2a_duOT2CeSov9nWEMR",
      Connectors.BluemixCloudObjectStorage.TARGET_BUCKET            : "projectibmib44b9b30d9304b75bfe3d588ecd1b28f",
      Connectors.BluemixCloudObjectStorage.TARGET_FILE_NAME         : "GoSalesTraining2.csv",
      Connectors.BluemixCloudObjectStorage.TARGET_WRITE_MODE        : "write",
      Connectors.BluemixCloudObjectStorage.TARGET_FILE_FORMAT       : "csv",
      Connectors.BluemixCloudObjectStorage.TARGET_FIRST_LINE_HEADER : "true"}

NewS3DF = df_r.write.format('com.ibm.spark.discover').options(**S3saveoptions).save()
df_r.show()

+--------------------+------+---+--------------+------------+------------+------------------+--------------------+----------------+---------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|GENDER_index|PRODUCT_LINE_index|MARITAL_STATUS_index|PROFESSION_index|AGE_index|
+--------------------+------+---+--------------+------------+------------+------------------+--------------------+----------------+---------+
|Personal Accessories|     M| 27|        Single|Professional|         0.0|               1.0|                 1.0|             1.0|      1.0|
|Personal Accessories|     F| 39|       Married|       Other|         1.0|               1.0|                 0.0|             0.0|      7.0|
|Mountaineering Eq...|     F| 39|       Married|       Other|         1.0|               2.0|                 0.0|             0.0|      7.0|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|         1.0|               1.0|                 2.0|             5.0|     38.0|
|     

In [56]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [57]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [58]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [59]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [40]:
train_data.printSchema()
train_data.show()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)

+-----------------+------+---+--------------+-----------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS| PROFESSION|
+-----------------+------+---+--------------+-----------+
|Camping Equipment|     F| 18|        Single|      Other|
|Camping Equipment|     F| 18|        Single|      Other|
|Camping Equipment|     F| 18|        Single|     Retail|
|Camping Equipment|     F| 18|        Single|     Retail|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|    

In [41]:
model_rf = pipeline_rf.fit(train_data)

NameError: name 'pipeline_rf' is not defined

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [62]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.589319
Test Error = 0.410681


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

First, you must import client libraries.

**Note**: Python 2 and Apache® Spark 2.0 or higher is required.

In [63]:
from repository_v3.mlrepositoryclient import MLRepositoryClient
from repository_v3.mlrepositoryartifact import MLRepositoryArtifact

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [64]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "mN3bwIsi/xpOh08Bd8D8Zah6UF++y/ZqlqTlJDmJQr6dqC83AFnAyPemGJw9js0JHxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv",
  "username": "75693a81-d133-4c15-8570-3dc0f0f092c1",
  "password": "13cb03fd-2b6b-4b24-a17f-49eb2efaf7b9",
  "instance_id": "366f855c-745d-4820-9b10-975dac91d5f7"
}

**Tip**: `wml_service_path`, user and `wml_password` can be found on **Service Credentials** tab of service instance created in Bluemix. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

In [65]:
ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

Create model artifact (abstraction layer).

In [66]:
pipeline_artifact = MLRepositoryArtifact(pipeline_rf, name="pipeline")

In [67]:
model_artifact = MLRepositoryArtifact(model_rf, training_data=train_data, name="Product Line Prediction PySpark", pipeline_artifact=pipeline_artifact)

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [68]:
saved_model = ml_repository_client.models.save(model_artifact)

Get saved model metadata from Watson Machine Learning.

**Tip**: Use *meta.available_props()* to get the list of available props.

In [69]:
saved_model.meta.available_props()

['inputDataSchema',
 'tags',
 'trainingDataReference',
 'modelVersionUrl',
 'creationTime',
 'runtimes',
 'trainingDataSchema',
 'trainingDefinitionVersionUrl',
 'version',
 'frameworkName',
 'hyperParameters',
 'contentStatus',
 'framework_runtimes',
 'label_column',
 'frameworkVersion']

In [70]:
print "modelType: " + saved_model.meta.prop("frameworkName")
print "creationTime: " + str(saved_model.meta.prop("creationTime"))

modelType: mllib
creationTime: 2018-04-06 20:38:39.074000+00:00


**Tip**: **modelVersionHref** is our model unique indentifier in the Watson Machine Learning repository.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [71]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print for example model name to make sure that model has been loaded correctly.

In [72]:
print str(loadedModelArtifact.name)

Product Line Prediction PySpark


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data using loaded model and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [73]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [74]:
predictions.show(5)

+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.54119162527029...|[0.27705958126351...|       1.0|Personal Accessories|
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.54119162527029...|[0.27705958126351...|       1.0|Personal Accessories|
|Camping E

By tabulating a count, you can see which product line is the most popular.

In [75]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

+------------------------+-----+
|predictedLabel          |count|
+------------------------+-----+
|Camping Equipment       |717  |
|Golf Equipment          |61   |
|Mountaineering Equipment|75   |
|Personal Accessories    |363  |
+------------------------+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install plotly --user <BR>
!pip install cufflinks==0.8.2 --user

Import Plotly and other required packages.

In [ ]:
!pip install plotly --user
!pip install cufflinks==0.8.2 --user
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [78]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [79]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()
product_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['GENDER'])]
product_layout = go.Layout(title='Predicted product line client interest distribution')

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [80]:
age_data = [go.Bar(y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"], x=cumulative_stats.index)]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(title = "Product Line", showline=False),
    yaxis=dict(title = "Mean AGE"))

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be over 50 years old.

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

In [81]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create online scoring endpoint

Now you can create an online scoring endpoint. 

#### Get published_models url from instance details

In [82]:
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)

print response_get_instance
print response_get_instance.text

<Response [200]>
{"metadata":{"guid":"366f855c-745d-4820-9b10-975dac91d5f7","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7","created_at":"2017-09-13T12:40:47.457Z","modified_at":"2018-04-06T20:38:39.318Z"},"entity":{"source":"Bluemix","published_models":{"url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7/published_models"},"usage":{"expiration_date":"2018-05-01T00:00:00.000Z","computation_time":{"limit":180000,"current":26413},"model_count":{"limit":200,"current":0},"prediction_count":{"limit":5000,"current":49},"gpu_count":{"limit":8,"current":0},"capacity_units":{"limit":180000000,"current":26413588},"deployment_count":{"limit":5,"current":1}},"plan_id":"3f6acf43-ede8-413a-ac69-f8af3bb0cbfe","status":"Active","organization_guid":"7b5f2512-63b7-4774-abe3-36df05119625","region":"us-south","account":{"id":"500107971","name":"BENOIT MAROLLEAU's Account","type":"TRIAL"},"owner":{"ibm_id":"06000

In [83]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print endpoint_published_models

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7/published_models


Execute the following sample code that uses the published_models endpoint to get deployments url.

#### Get the list of published models

In [ ]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get = requests.get(endpoint_published_models, headers=header)

print response_get
print response_get.text

#### Get published model deployment url

In [85]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print endpoint_deployments

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7/published_models/8a0efc52-3bea-49f8-91d2-8819fa9723c6/deployments


#### Create online deployment for published model

In [86]:
payload_online = {"name": "Product Line Prediction PySparkDeploy", "description": "My Cool Deployment", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

print response_online
print response_online.text

<Response [201]>
{"metadata":{"guid":"f36d5482-f826-4e9f-864d-41618989a61d","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7/published_models/8a0efc52-3bea-49f8-91d2-8819fa9723c6/deployments/f36d5482-f826-4e9f-864d-41618989a61d","created_at":"2018-04-06T20:43:45.141Z","modified_at":"2018-04-06T20:43:48.714Z"},"entity":{"runtime_environment":"spark-2.0","name":"Product Line Prediction PySparkDeploy","scoring_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7/published_models/8a0efc52-3bea-49f8-91d2-8819fa9723c6/deployments/f36d5482-f826-4e9f-864d-41618989a61d/online","description":"My Cool Deployment","published_model":{"author":{},"name":"Product Line Prediction PySpark","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7/published_models/8a0efc52-3bea-49f8-91d2-8819fa9723c6","guid":"8a0efc52-3bea-49f8-91d2-8819fa9723c6","created_at":"2018-04-06T20:4

In [87]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')
print scoring_url

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/366f855c-745d-4820-9b10-975dac91d5f7/published_models/8a0efc52-3bea-49f8-91d2-8819fa9723c6/deployments/f36d5482-f826-4e9f-864d-41618989a61d/online


Now, you can send (POST) new scoring records (new data) for which you would like to get predictions. To do that, execute the following sample code: 

In [88]:
payload_scoring = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)

print response_scoring.text

{
  "fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "PRODUCT_LINE", "label", "PROFESSION_IX", "GENDER_IX", "MARITAL_STATUS_IX", "features", "rawPrediction", "probability", "prediction", "predictedLabel"],
  "values": [["M", 23, "Single", "Student", "Camping Equipment", 0.0, 6.0, 0.0, 1.0, [0.0, 23.0, 1.0, 6.0], [5.600716147152702, 6.482458372136143, 6.048004170730676, 0.20929155492307386, 1.6595297550574055], [0.2800358073576351, 0.32412291860680714, 0.3024002085365338, 0.010464577746153694, 0.08297648775287028], 1.0, "Personal Accessories"], ["M", 55, "Single", "Executive", "Camping Equipment", 0.0, 3.0, 0.0, 1.0, [0.0, 55.0, 1.0, 3.0], [6.227653744886563, 4.325198862164969, 8.031953760146816, 1.2319759269281225, 0.1832177058735289], [0.3113826872443282, 0.2162599431082485, 0.40159768800734086, 0.06159879634640614, 0.009160885293676447], 2.0, "Mountaineering Equipment"]]
}


As we can see we predict that a 23-year-old male student is interested in personal accessories (predictedLabel: Personal Accessories, prediction: 1.0). We can also see that single, 55 years old man is interested in mountaineering equipment.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

**Updated: Benoit Marolleau - IBM France, Cloud Architect - with Connections & object storage API 

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.